# Assignment 01

- Name: Ran Arino
- Student ID: 153073200
- Email: rarino@myseneca.ca
- Course: Health Analytics
- Course ID: BDA700NAA.05841.2247

- Data source: https://www.kaggle.com/datasets/trainingdatapro/brain-mri-dataset